In [ ]:
import pandas as pd
from hashlib import blake2b
import re
import pycountry
import numpy as np
import json



df = pd.read_csv("../data/community_builders_raw.csv").drop(columns=["cbName", "cbBioURL", "dateCreated", "dateUpdated"]) 

# anonymize builder id
df['id'] = df['id'].apply(lambda x: blake2b(str(x).encode(), digest_size=10).hexdigest())
df.head()

In [ ]:
# getting 3 columns from cbCategory: category, year and the phrase
# category will be used for categorization while year will be for QC

df[['Category', 'phrase', 'year_since']] = df['cbCategory'].str.extract(
    r'(.*?)\s*(Builder|since|Builder since)\s*(\d{4})',
    flags=re.IGNORECASE
)
df['Category'] = df['Category'].str.strip()

In [ ]:
# check 1 if the process what successful
df[df['Category'].isna()]

In [ ]:
# check 2 if the process what successful
df['year_since'].unique()

In [ ]:
# check 3 if the process what successful
df['phrase'].unique()

In [ ]:
# check categories
df['Category'].unique()

In [ ]:
# fix the categories
df['Category'] = df['Category'].replace(
    {
        'GameTech': 'Game Tech',
        'Network Content & Deliver': 'Networking & Content Delivery'
    }
)

# recheck categories
df['Category'].unique()

In [ ]:
# check for missing year in the original year cols
df['year'].unique()

In [ ]:
df[df['year'].isna()]

In [ ]:
# check for missing year in the extracted year cols 
df['year_since'].unique()

In [ ]:
# drop columns after cleaning and checks
drop_cols_2 = ['cbCategory', 'year', 'phrase']
df = df.drop(columns=drop_cols_2)

In [ ]:
df[df['country'].isna()]

In [ ]:
# Update country for specific rows (indices 205, 276, 596)
df.loc[[205, 276, 596], 'country'] = df.loc[[205, 276, 596], 'cbLocation'].str.split(',').str[1].str.strip()
df.iloc[[205, 276, 596]]

In [ ]:
# validate the countries present
df['country'] = df['country'].str.strip()
countries_ = [*df['country'].unique()]
len(countries_)

In [ ]:
df[df['country'].isna()]

In [ ]:
pyc_countries = set(country.name for country in pycountry.countries)

In [ ]:
np.array([c for c in countries_ if c not in pyc_countries])

In [ ]:
invalid_countries = ['Europe, Middle East, & Africa', 'Asia Pacific', 
                     'Central & Eastern Europe', 'Korea', "Northern Ireland", 
                     'Americas', 'Latin America', 'Europe', 'UK/IR', 'North America']
df_val_countries = df[~df['country'].isin(invalid_countries)]
filtered_df_invalid_countries = df[df['country'].isin(invalid_countries)]

filtered_df_invalid_countries.head()

In [ ]:
def extract_country(location):
    if pd.isna(location):
        return None
    if ',' in location:
        return location.split(',')[-1].strip()
    return location.strip()

filtered_df_invalid_countries['country_'] = filtered_df_invalid_countries['cbLocation'].apply(extract_country)


In [ ]:
filtered_df_invalid_countries.head()

In [ ]:
filtered_df_invalid_countries['country_'].unique()

In [ ]:
# Dictionary for correcting country names
country_corrections = {
    'Aoterora': 'New Zealand', 
    'Sweden and Spain': 'Spain',
    'UK': 'United Kingdom', 
    'Korea': 'South Korea',
    'UK&I': 'United Kingdom',
    'Northern Ireland': 'United Kingdom',
    'Scotland': 'United Kingdom', 
    'United States of America': 'United States',
    'St. Lucia': 'Saint Lucia',
    'SriLanka': 'Sri Lanka',
    'Somaliland': 'Somalia', 
    'Kurdistan': 'Turkey',  
    'Republic of Ireland': 'Ireland',
    'Republic of Moldova': 'Moldova'
}

# Apply corrections to the country column
filtered_df_invalid_countries['country_'] = filtered_df_invalid_countries['country_'].replace(country_corrections)

In [ ]:
filtered_df_invalid_countries['country_'].unique()

In [ ]:
df_val_countries.columns

In [ ]:
df_val_countries.shape

In [ ]:
df_val_countries_2 = filtered_df_invalid_countries \
    .drop(columns=['country']) \
    .rename(columns={'country_': 'country'})

In [ ]:
df_val_countries_2.columns

In [ ]:
# Concatenate the DataFrames
result = pd.concat([df_val_countries, df_val_countries_2], ignore_index=True)
result.head()

In [ ]:
result['country'].unique()

In [ ]:
result['region'].unique()

In [ ]:
with open('../data/country_regions.json', 'r') as file:
    country_region_mapping = json.load(file)
    
result['region_'] = result['country'].map(country_region_mapping)
result.head()

In [ ]:
result['region_'].unique()

In [ ]:
result[result['region_'].isna()]

In [ ]:
df = result.drop(columns=['region', 'cbLocation']).rename(
    columns= {
        "id": "ID",
        "country": "Country",
        "region_": "Region",
        "year_since": "Year"
    }
)

In [ ]:
df.to_csv('community_builders.csv', index=False)